# General preparations

In [1]:
### Preparations
import pandas as pd
import numpy as np
from openai import OpenAI
import json
import time
import os
# API key for OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
  api_key=openai_api_key
)

In [2]:
# Define a function to split tasks into a maximum of two batches
def split_into_batches(tasks, batch_size=450):
    if len(tasks) <= batch_size:
        return [tasks]
    return [tasks[:batch_size], tasks[batch_size:batch_size*2]]

# Data preparations

In [1]:
### load data from reviews_general table
data = pd.read_csv("reviews_general_selected.csv") # as an example

# preprocess the data
data = data.dropna(subset=['review_text']) # drop rows with missing review_text
data['review_text'] = data['review_text'].str.replace(r'\s+', ' ', regex=True).str.strip() # remove extra spaces, newlines, and tabs
data = data[data["review_text"].str.len() > 10] # drop rows where the "review_text" column contains less than 10 characters

NameError: name 'pd' is not defined

In [4]:
### for testing purposes, we will only use the first 1000 reviews
data = data.head(1000)

category_df = pd.read_csv("category_df.csv")
# keep only reviews in category_df that are in data
category_df = category_df[category_df["review_id"].isin(data["review_id"])]

In [5]:
# Function to save each batch to a separate .jsonl file
def save_batches_to_jsonl(batches, prefix):
    batch_file_names = []
    
    for i, batch in enumerate(batches):
        batch_file_name = f"{prefix}_part{i+1}.jsonl"
        with open(batch_file_name, 'w') as file:
            for task in batch:
                file.write(json.dumps(task) + '\n')
        batch_file_names.append(batch_file_name)
    
    return batch_file_names  # Return the list of saved file names

In [6]:
# Function to upload batch files to the API
def upload_batch_files(batch_file_names):
    batch_file_ids = []
    
    for batch_file_name in batch_file_names:
        batch_file = client.files.create(
            file=open(batch_file_name, "rb"),
            purpose="batch"
        )
        print(f"Batch file uploaded: {batch_file.id}")
        batch_file_ids.append(batch_file.id)
    
    return batch_file_ids  # Return the list of uploaded file IDs

In [7]:
# Function to start batch jobs
def start_batch_jobs(batch_file_ids):
    batch_job_ids = []
    
    for file_id in batch_file_ids:
        batch_job = client.batches.create(
            input_file_id=file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h"
        )
        print(f"Batch job started: {batch_job.id}")
        batch_job_ids.append(batch_job.id)
    
    return batch_job_ids  # Return the list of batch job IDs

In [8]:
# Function to check batch job status
def wait_for_batches_to_complete(batch_job_ids):
    while True:
        all_done = True
        for batch_job_id in batch_job_ids:
            batch_job = client.batches.retrieve(batch_job_id)
            status = batch_job.status
            print(f"Batch Job {batch_job_id} Status: {status}")

            if status != 'completed':
                all_done = False  # Keep waiting if any batch is still running

        if all_done:
            print("All batch jobs are completed.")
            break  # Exit loop when all batches are done
        
        print(f"Waiting 60 seconds before checking again...")
        time.sleep(60)  # Wait before checking again

# 5. Sentiment Analysis

In [12]:
# Define the system prompt for sentiment analysis
sentiment_prompt = (
    "Rate the sentiment of the following sentences on a scale of 1 to 5, "
    "where 1 is 'very bad' and 5 is 'very good'. Return the result as a JSON object with the key 'rating'. "
    "Only include the JSON object in your response.\n\n"
    "Sentences: {sentences}\n\n"
    "JSON:"
)

# Define a function to create batch tasks for any category
def create_sentiment_batch(data, category):
    tasks = []
    category_column = f"{category}_sentences"

    for index, row in data.iterrows():
        sentences = row[category_column]
        review_id = row['review_id']  # Access the review_id

        if not sentences or pd.isna(sentences):  # Skip empty or NaN sentences
            continue

        task = {
            "custom_id": f"{review_id}_{category}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o-mini-2024-07-18",
                "temperature": 0.1,
                "response_format": {
                    "type": "json_object"
                },
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a sentiment analysis expert specializing in restaurant reviews."
                    },
                    {
                        "role": "user",
                        "content": sentiment_prompt.format(sentences=sentences)
                    }
                ],
            }
        }
        tasks.append(task)
    
    return tasks

## 5.1 Food

In [14]:
# Prepare the batch tasks as a list
food_tasks = create_sentiment_batch(category_df, "food")

# Split tasks into batches of 45000 each
batches_food = split_into_batches(food_tasks, batch_size=450)

batch_file_names_food = save_batches_to_jsonl(batches_food, prefix="batch_tasks_food")
batch_file_ids_food = upload_batch_files(batch_file_names_food)
batch_job_ids_food = start_batch_jobs(batch_file_ids_food)

Batch file uploaded: file-19NQv2a6EMcb4M3SDwBAaz
Batch file uploaded: file-1UK95j37Nh244pMCsc5tac
Batch job started: batch_679a626682ec8190919822251742d98c
Batch job started: batch_679a62675a7081909f26e0af5a8e45bb


## 5.2 Service

In [15]:
# Prepare the batch tasks as a list
service_tasks = create_sentiment_batch(category_df, "service")

# Split tasks into batches of 45000 each
batches_service = split_into_batches(service_tasks, batch_size=450)

batch_file_names_service = save_batches_to_jsonl(batches_service, prefix="batch_tasks_service")
batch_file_ids_service = upload_batch_files(batch_file_names_service)
batch_job_ids_service = start_batch_jobs(batch_file_ids_service)

Batch file uploaded: file-D2dsz9WnvDtaEZWkPRigPp
Batch job started: batch_679a626ed55c819080c578ee845c4a29


## 5.3 Atmosphere

In [16]:
# Prepare the batch tasks as a list
atmosphere_tasks = create_sentiment_batch(category_df, "atmosphere")

# Split tasks into batches of 45000 each
batches_atmosphere = split_into_batches(atmosphere_tasks, batch_size=450)

batch_file_names_atmosphere = save_batches_to_jsonl(batches_atmosphere, prefix="batch_tasks_atmosphere")
batch_file_ids_atmosphere = upload_batch_files(batch_file_names_atmosphere)
batch_job_ids_atmosphere = start_batch_jobs(batch_file_ids_atmosphere)

Batch file uploaded: file-HrzsAwKWkqiANFv4V5r3tc
Batch job started: batch_679a6341d9b08190baa02406db3d3943


In [17]:
# Collect all batch job IDs into one list
batch_job_ids_sentiment = batch_job_ids_food + batch_job_ids_service + batch_job_ids_atmosphere

# 6. Category summaries

In [18]:
FOOD_SUMMARY_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about the food in a restaurant. "
    "The summary should focus exclusively on customer perceptions of the food, including aspects like taste, presentation, freshness, and variety. "
    "Do not include information about price, service, or atmosphere. "
    "List up to the 5 most positively recommended items in a second section as bullet points. "
    "Only include food items in the recommendations section if customers mention them positively. "
    "Do not list items with mixed or negative reviews. "
    "Write concisely, strictly in English and limit the overall response to around 400 characters.\n\n"
    "Reviews:\n{reviews}\n\n"
    "Summary:"
)

FOOD_COMBINE_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about food. Combine the following two summaries into a single cohesive summary in English. "
    "The combined summary should focus exclusively on customer perceptions of the food, including aspects like taste, presentation, freshness, and variety. "
    "Do not include information about price, service, or atmosphere. "
    "List up to the 5 most positively recommended items in a second section as bullet points. "
    "Only include food items in the recommendations section if customers mention them positively. "
    "Do not list items with mixed or negative reviews. "
    "Limit the overall response to around 400 characters:\n\n"
    "Summary 1:\n{summary1}\n\n"
    "Summary 2:\n{summary2}\n\n"
    "Combined Summary:"
)

SERVICE_SUMMARY_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about the service in restaurants. "
    "The summary should focus exclusively on customer perceptions of the service, including aspects like speed, attentiveness, friendliness, and professionalism. "
    "Do not include information about price, food, or atmosphere. "
    "Write concisely, strictly in English and limit the overall response to around 400 characters.\n\n"
    "Reviews:\n{reviews}\n\n"
    "Summary:"
)

SERVICE_COMBINE_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about service in restaurants. Combine the following two summaries into a single cohesive summary in English."
    "The combined summary should focus exclusively on customer perceptions of the service, including aspects like speed, attentiveness, friendliness, and professionalism. "
    "Do not include information about price, food, or atmosphere. "
    "Limit the overall response to around 400 characters:\n\n"
    "Summary 1:\n{summary1}\n\n"
    "Summary 2:\n{summary2}\n\n"
    "Combined Summary:"
)

ATMOSPHERE_SUMMARY_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about the atmosphere in restaurants. "
    "The summary should focus exclusively on customer perceptions of the atmosphere, including aspects like ambiance, decor, cleanliness, noise levels, and overall vibe. "
    "Do not include information about price, food, or service. "
    "Write concisely, strictly in English and limit the overall response to around 400 characters.\n\n"
    "Reviews:\n{reviews}\n\n"
    "Summary:"
)

ATMOSPHERE_COMBINE_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about the atmosphere in restaurants. Combine the following two summaries into a single cohesive summary in English. "
    "The combined summary should focus exclusively on customer perceptions of the atmosphere, including aspects like ambiance, decor, cleanliness, noise levels, and overall vibe. "
    "Do not include information about price, food, or service. "
    "Limit the overall response to around 400 characters:\n\n"
    "Summary 1:\n{summary1}\n\n"
    "Summary 2:\n{summary2}\n\n"
    "Combined Summary:"
)

PRICE_SUMMARY_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about pricing in restaurants. "
    "The summary should focus exclusively on customer perceptions of the price, including aspects like value for money, affordability, and pricing fairness. "
    "Do not include information about food, service, or atmosphere. "
    "Write concisely, strictly in English and limit the overall response to around 400 characters.\n\n"
    "Reviews:\n{reviews}\n\n"
    "Summary:"
)

PRICE_COMBINE_PROMPT = (
    "You are an expert summarizer specializing in customer opinions about pricing in restaurants. Combine the following two summaries into a single cohesive summary in English. "
    "The combined summary should focus exclusively on customer perceptions of the price, including aspects like value for money, affordability, and pricing fairness. "
    "Do not include information about food, service, or atmosphere. "
    "Limit the overall response to around 400 characters:\n\n"
    "Summary 1:\n{summary1}\n\n"
    "Summary 2:\n{summary2}\n\n"
    "Combined Summary:"
)

In [20]:
# Function to summarize a single chunk of reviews using modular prompts
def summarize_chunk(prompt, reviews_chunk):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt.format(reviews=reviews_chunk)},
            ],
            max_tokens=200,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error summarizing chunk: {e}")
        return None
    

# Function to combine summaries using modular prompts
def combine_summaries(combine_prompt, summary1, summary2):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": combine_prompt.format(summary1=summary1, summary2=summary2)},
            ],
            max_tokens=200,
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error combining summaries: {e}")
        return f"{summary1}\n\n{summary2}"

    

# Function to handle larger price reviews, retry, and combine summaries
def summarize_reviews(restaurant_id, reviews_df, category_column_name, summary_prompt, combine_prompt):
    """
    Generalized function to summarize reviews for a specific aspect (overall, service, atmosphere, etc.).
    
    Args:
        restaurant_id (int): ID of the restaurant.
        reviews_df (DataFrame): DataFrame containing review data.
        category_column_name (str): Column in the DataFrame containing the reviews for this aspect.
        summary_prompt (str): Prompt for summarizing reviews.
        combine_prompt (str): Prompt for combining summaries.
    
    Returns:
        str: Final summarized review.
    """
    # Filter and join the reviews for the specified column
    reviews = reviews_df[reviews_df['restaurant_id'] == restaurant_id][category_column_name].str.strip().replace(["", "nan", "None", "null"], pd.NA).dropna().astype(str).tolist()
    
    # If no reviews exist, return None
    if not reviews:
        return None, 0  # No summary, zero review count
    
    # Join the reviews into a single text chunk
    reviews_text = "\n".join(reviews)

    # Getting amount of reviews
    count_reviews = len(reviews)
    
    try:
        # Attempt to summarize the full chunk
        return summarize_chunk(summary_prompt, reviews_text), count_reviews
    
    except Exception as e:
        if "context_length_exceeded" in str(e):
            print(f"Context length exceeded for restaurant '{restaurant_id}' in column '{category_column_name}'. Splitting reviews...")
            
            # Split reviews into halves
            mid_point = len(reviews) // 2
            first_half = "\n".join(reviews[:mid_point])
            second_half = "\n".join(reviews[mid_point:])
            
            # Summarize each half
            first_summary = summarize_chunk(summary_prompt, first_half)
            second_summary = summarize_chunk(summary_prompt, second_half)
            
            if first_summary and second_summary:
                # Combine summaries
                try:
                    return combine_summaries(combine_prompt, first_summary, second_summary), count_reviews
                except Exception as combine_error:
                    print(f"Error combining summaries for restaurant '{restaurant_id}' in column '{category_column_name}': {combine_error}")
                    return f"{first_summary}", (len(reviews) // 2)  # Return the first summary if combining fails
            else:
                return f"{first_summary or 'Error in first half'}\n\n{second_summary or 'Error in second half'}", (len(reviews) // 2)
        else:
            print(f"Error summarizing reviews for restaurant '{restaurant_id}' in column '{category_column_name}': {e}")
            return None, None

In [19]:
# merge the categorized sentences with reviews general to get the restaurant_ids
reviews_df = pd.merge(category_df, data, on='review_id', how='left')
# keep only the necessary columns
reviews_df = reviews_df[['review_id', 'restaurant_id', 'food_sentences', 'service_sentences', 'atmosphere_sentences', 'price_sentences']]

In [21]:
# Extract unique restaurant IDs of the two batches
unique_restaurant_ids = reviews_df['restaurant_id'].dropna().unique()

summaries_categories = []  # List to store summaries

# Generate summaries for each restaurant
for restaurant_id in unique_restaurant_ids:

    # Summarize food reviews
    food_summary, user_count_food = summarize_reviews(restaurant_id, reviews_df, 'food_sentences', FOOD_SUMMARY_PROMPT, FOOD_COMBINE_PROMPT)
    
    # Summarize service reviews
    service_summary, user_count_service = summarize_reviews(restaurant_id, reviews_df, 'service_sentences', SERVICE_SUMMARY_PROMPT, SERVICE_COMBINE_PROMPT)
    
    # Summarize atmosphere reviews
    atmosphere_summary, user_count_atmosphere = summarize_reviews(restaurant_id, reviews_df, 'atmosphere_sentences', ATMOSPHERE_SUMMARY_PROMPT, ATMOSPHERE_COMBINE_PROMPT)
    
    # Summarize price reviews
    price_summary, user_count_price = summarize_reviews(restaurant_id, reviews_df, 'price_sentences', PRICE_SUMMARY_PROMPT, PRICE_COMBINE_PROMPT)
    
    # Append the summaries to the list
    summaries_categories.append({
        "restaurant_id": restaurant_id,
        "summary_food": food_summary,
        "summary_service": service_summary,
        "summary_atmosphere": atmosphere_summary,
        "summary_price": price_summary,
        "user_count_food": user_count_food,
        "user_count_service": user_count_service,
        "user_count_atmosphere": user_count_atmosphere,
        "user_count_price": user_count_price,
    })

# Convert the list of summaries into a DataFrame
summaries_categories_df = pd.DataFrame(summaries_categories)

# Display the resulting DataFrame
print(summaries_categories_df.head())

C:\Users\julia\AppData\Local\Temp\ipykernel_23208\1505715936.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  reviews = reviews_df[reviews_df['restaurant_id'] == restaurant_id][category_column_name].str.strip().replace(["", "nan", "None", "null"], pd.NA).dropna().astype(str).tolist()
C:\Users\julia\AppData\Local\Temp\ipykernel_23208\1505715936.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  reviews = reviews_df[reviews_df['restaurant_id'] == restaurant_id][category_column_name].str.strip().replace(["", "nan", "None", "nul

                 restaurant_id  \
0  ChIJ_VWb4xn6mUcRH4NujtHMKJI   
1  ChIJo5EYOK_4mUcRi4shjNiEDUc   
2  ChIJ_VfiMxj6mUcRRK_QBdxww7g   
3  ChIJT4FlA7zwmUcRcrmR1JIlwm8   
4  ChIJ5dhz-EXxmUcRuZTn4wsBpQs   

                                        summary_food  \
0  Customers generally appreciate the food qualit...   
1                                               None   
2  Customers consistently praise the food for its...   
3  Customers consistently praise the food at Naza...   
4  Customers overwhelmingly praise the food for i...   

                                     summary_service  \
0  Customer perceptions of service vary significa...   
1  Customers consistently praise the service, hig...   
2  Customers consistently praise the service at t...   
3  Customers consistently perceive the service as...   
4  Customer perceptions of service vary significa...   

                                  summary_atmosphere  \
0  Customers consistently describe the atmosphere...   
1  Custom

In [24]:
# save batch_job_ids_sentiment
with open('batch_job_ids_sentiment.txt', 'w') as f:
    for item in batch_job_ids_sentiment:
        f.write("%s\n" % item)

# 7. Transition - wait unitl 5. is finished

In [26]:
# Function to check batch job status, to wait for completion
def wait_for_batches_to_complete(batch_job_ids):
    while True:
        completed_batches = 0
        failed_batches = 0
        total_batches = len(batch_job_ids)

        # Track failed batch job IDs
        failed_batch_ids = []

        for batch_job_id in batch_job_ids:
            batch_job = client.batches.retrieve(batch_job_id)
            status = batch_job.status
            print(f"Batch Job {batch_job_id} Status: {status}")

            if status == "completed":
                completed_batches += 1
            elif status in ["failed", "cancelled"]:
                failed_batches += 1
                failed_batch_ids.append(batch_job_id)  # Store failed job ID

        # If all batches are completed, proceed
        if completed_batches == total_batches:
            print("All batch jobs are completed.")
            return  # Exit the function

        # If all batches have failed
        if failed_batches == total_batches:
            print("All batch jobs have failed. Waiting for 1 hour...")
     
            return  # After waiting, return so the next batch can be processed

        # Only exit if all batches are either completed or failed
        if completed_batches + failed_batches == total_batches:
            print("All batch jobs are either completed or failed. Exiting wait function.")
            # Update batches_protocol with the failed job IDs if any
            return  # Exit the function

        # Wait before checking again
        print(f"Waiting 60 seconds before checking again...")
        time.sleep(60)

In [27]:
# Wait for all batch jobs to complete
wait_for_batches_to_complete(batch_job_ids_sentiment)

Batch Job batch_679a626682ec8190919822251742d98c Status: in_progress
Batch Job batch_679a62675a7081909f26e0af5a8e45bb Status: completed
Batch Job batch_679a626ed55c819080c578ee845c4a29 Status: completed
Batch Job batch_679a6341d9b08190baa02406db3d3943 Status: completed
Waiting 60 seconds before checking again...
Batch Job batch_679a626682ec8190919822251742d98c Status: in_progress
Batch Job batch_679a62675a7081909f26e0af5a8e45bb Status: completed
Batch Job batch_679a626ed55c819080c578ee845c4a29 Status: completed
Batch Job batch_679a6341d9b08190baa02406db3d3943 Status: completed
Waiting 60 seconds before checking again...
Batch Job batch_679a626682ec8190919822251742d98c Status: in_progress
Batch Job batch_679a62675a7081909f26e0af5a8e45bb Status: completed
Batch Job batch_679a626ed55c819080c578ee845c4a29 Status: completed
Batch Job batch_679a6341d9b08190baa02406db3d3943 Status: completed
Waiting 60 seconds before checking again...
Batch Job batch_679a626682ec8190919822251742d98c Status: i

# 8. Retrieve subratings

In [28]:
def retrieve_batch_results_subratings(batch_job_ids, category):
    results = []

    for batch_job_id in batch_job_ids:
        # Get batch job status
        batch_job_status = client.batches.retrieve(batch_job_id)
        result_file_id = batch_job_status.output_file_id

        # ! think about error handeling !
        if not result_file_id:
            print(f"No output file found for batch job {batch_job_id}")
            continue  # Skip if there's no result file

        # Download the batch results file
        result_content = client.files.content(result_file_id).content
        result_file_name = f"batch_results_{category}.jsonl"

        with open(result_file_name, 'wb') as file:
            file.write(result_content)

        # Read and parse the results
        with open(result_file_name, 'r') as file:
            for line in file:
                try:
                    response = json.loads(line.strip())
                    custom_id = response['custom_id']  # Format: review_id_category
                    review_id, _ = custom_id.rsplit("_", 1)  # Extract review_id
                    
                    # Extract rating from response
                    body = response.get("response", {}).get("body", {})
                    choices = body.get("choices", [])
                    if choices:
                        content = json.loads(choices[0]["message"]["content"])
                        rating = content.get('rating', None)
                    else:
                        rating = None

                    # Append results
                    results.append({"review_id": int(review_id), f"rating_{category}": rating})

                except Exception as e:
                    print(f"Error parsing result for {category}: {e}")

    return pd.DataFrame(results)

In [29]:
# Retrieve results for each category
df_food = retrieve_batch_results_subratings(batch_job_ids_food, "food")
df_service = retrieve_batch_results_subratings(batch_job_ids_service, "service")
df_atmosphere = retrieve_batch_results_subratings(batch_job_ids_atmosphere, "atmosphere")

# Merge all results into a single DataFrame
df_ratings = df_food.merge(df_service, on="review_id", how="outer").merge(df_atmosphere, on="review_id", how="outer")

# Display the final DataFrame
print(df_ratings.head())

Error parsing result for food: Expecting property name enclosed in double quotes: line 2 column 1 (char 2)
Error parsing result for service: Unterminated string starting at: line 2 column 3 (char 4)
   review_id  rating_food  rating_service  rating_atmosphere
0       4471          5.0             1.0                NaN
1       4472          4.0             NaN                5.0
2       4473          1.0             NaN                NaN
3       4474          NaN             NaN                5.0
4       4475          NaN             1.0                NaN
